In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import copy

# --- GPU利用設定 ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- CIFAR-10用のtransform ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet用にサイズ変更
    transforms.ToTensor()
])

# --- CIFAR-10のテストデータ ---
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=True)

# --- ResNet18のロード（ImageNet事前学習済み） ---
model = resnet18(pretrained=True).to(device)
model.eval()

# --- PGD攻撃関数 ---
def pgd_attack(model, images, labels, eps=0.03, alpha=0.005, iters=10):
    images = images.clone().detach().to(device)
    ori_images = images.clone().detach()
    labels = labels.to(device)

    for i in range(iters):
        images.requires_grad = True
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        model.zero_grad()
        loss.backward()
        grad = images.grad.data

        adv_images = images + alpha * grad.sign()
        eta = torch.clamp(adv_images - ori_images, min=-eps, max=eps)
        images = torch.clamp(ori_images + eta, min=0, max=1).detach()

    return images

# --- 攻撃実行テスト ---
data_iter = iter(testloader)
images, labels = next(data_iter)

# CIFAR-10は10クラスなので、ImageNetの1000クラスモデルに合わせるには以下のようにマッピング調整が必要ですが、
# 今回はシンプルに「ラベルを無視して予測の変化だけ確認」でもOKです。

images = images.to(device)
outputs_clean = model(images)
pred_clean = outputs_clean.argmax(dim=1)

# 攻撃後画像
adv_images = pgd_attack(model, images, labels, eps=0.03, alpha=0.005, iters=10)
outputs_adv = model(adv_images)
pred_adv = outputs_adv.argmax(dim=1)

print("元画像の予測ラベル：", pred_clean.item())
print("攻撃画像の予測ラベル：", pred_adv.item())


100%|██████████| 170M/170M [00:27<00:00, 6.30MB/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data


C:\Users\flare\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\flare\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\flare/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:06<0

元画像の予測ラベル： 906
攻撃画像の予測ラベル： 680
